In [1]:
import cobra
from cobra import Model, Reaction, Metabolite
import pandas as pd
import cplex
import optlang
import math
import json
import sys
import os
import cProfile
from optlang.symbolics import Zero, add
import cobra.util.solver as sutil
from cobra.flux_analysis.parsimonious import pfba
from IPython.core.display import HTML
from configparser import ConfigParser
config = ConfigParser()
#os.chdir('/Users/chenry/code/ProjectNotebooks/SynBioSFA')
config.read("config.cfg")
paths = config.get("script", "syspaths").split(";")
for path in paths:
    sys.path.append(path)
from modelseedpy import ElementUptakePkg, SimpleThermoPkg, ReactionUsePkg, BilevelPkg

In [2]:
model= cobra.io.read_sbml_model("../../iML1515.xml")
model.solver = 'optlang-cplex'
medium = {
         'EX_so4_e':100,
         'EX_pi_e':100,
         'EX_mn2_e': 100,
         'EX_fe2_e': 100,
         'EX_na1_e': 100,
         'EX_mg2_e': 100,
         'EX_nh4_e': 100,
         'EX_k_e': 100,
         'EX_o2_e': 20,
         'EX_h2o_e': 100,
         'EX_h_e': 100,
         'EX_fe3_e': 100.0,
         'EX_zn2_e': 100.0,
         'EX_ca2_e': 100.0,
         'EX_ni2_e': 100.0,
         'EX_cu2_e': 100.0,
         'EX_sel_e': 100.0,
         'EX_cobalt2_e': 100.0,
         'EX_mobd_e': 100.0,
         'EX_cl_e': 100.0,
         'EX_cbl1_e':100,
         'EX_tungs_e': 100.0,
         'EX_slnt_e': 100.0
}
model.medium=medium

biomass_objective = model.problem.Objective(
    1 * model.reactions.BIOMASS_Ec_iML1515_WT_75p37M.flux_expression,
    direction='max')
model.objective = biomass_objective



In [3]:
carbon_sources=["EX_malt_e","EX_g1p_e","EX_f6p_e", "EX_dad_2_e","EX_glyclt_e" ,"EX_acnam_e", "EX_gam_e","EX_lac__D_e", "EX_melib_e"]
ignore=['EX_co2_e','EX_h_e','EX_h2o_e']
# not in model "EX_2hb_e"

In [7]:

def max_export():
    exports={}
    
    sol1=model.optimize()
    if sol1.status is 'infeasible':
        return ['inf']
    else:
        for rxn in sol1.fluxes.index:
            if sol1.fluxes[rxn]>0 and "EX_" in rxn and rxn not in ignore:
                    exports[rxn]=sol1.fluxes[rxn]

        max_keys = [key for key, value in exports.items() if value == max(exports.values())]
        max_keys.append('test')
        max_key=max_keys[0]
        max_val = exports.get(max_key)
        bio=sol1.fluxes["BIOMASS_Ec_iML1515_WT_75p37M"]
        return [max_key, max_val,bio];


results=[['carbon source ID','carbon source', 'simulation id' ,'simulation', 'biomass', 'largest byproduct', 'byproduct id', 'amount of byproduct']]
for csid in carbon_sources:
    print(csid)
    with model:
        model.reactions.get_by_id(csid).lower_bound=-5
        simid= 'WT KO'
        sim='wild type'
        res_list = max_export()
        if res_list[0] is 'inf':
            print('inf')
            break
        else:
            #print(res_list)
            max_keyid=res_list[0]
            max_key=model.reactions.get_by_id(max_keyid).name
            max_value=res_list[1]
            bio=res_list[2]
            cs=model.reactions.get_by_id(csid).name
            line=[csid,cs, simid,sim, bio,max_key,max_keyid,max_value]
            results.append(line)
        
        for i in range(1,15):
            #print (max_keyid)
            model.reactions.get_by_id(max_keyid).knock_out()
            simid= max_keyid +' KO'
            sim=model.reactions.get_by_id(max_keyid).name
            res_list = max_export()
            if res_list[0] is 'inf' or res_list[0] is 'test':
                print(res_list)
                break
            else:
                print(res_list)
                max_keyid=res_list[0]
                max_key=model.reactions.get_by_id(max_keyid).name
                max_value=res_list[1]
                bio=res_list[2]
                line=[csid,cs, simid,sim, bio,max_key,max_keyid,max_value]
                results.append(line)
            
results_df=pd.DataFrame(results)

results_df

EX_malt_e
['EX_etoh_e', 1.82213353179759, 0.8038702002637702]
['EX_lac__D_e', 1.8221335317973626, 0.803870200263708]
['EX_hxa_e', 0.6890327620981891, 0.8028026401209905]
['EX_acald_e', 2.2160600961542514, 0.8021534622777381]
['EX_pyr_e', 2.2160600961541377, 0.8021534622777379]
['EX_ala__L_e', 1.9320727228783792, 0.7948449138289996]
['EX_leu__L_e', 0.7707338779584495, 0.7948449138290017]
['EX_val__L_e', 0.9634173474480576, 0.794844913828999]
['EX_succ_e', 1.6686074856094886, 0.7934600209910487]
['EX_4abut_e', 1.34762017507435, 0.7881305082048196]
['EX_ala__D_e', 0.02327036472900994, 0.7845177239906257]
['EX_5mtr_e', 0.005289754224438673, 0.7843645054031247]
['EX_for_e', 0.6408654407452967, 0.0]
['EX_enlipa_e', 0.028886526099631424, 0.0]
EX_g1p_e


cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'


['inf']
EX_f6p_e
['inf']
EX_dad_2_e
['EX_alltn_e', 4.79727216371119, 0.4613500801936391]
['EX_ade_e', 4.0943514035681545, 0.4300699835008709]
['EX_xan_e', 3.5467896445278484, 0.4216346777736384]
['EX_gua_e', 4.14079133402409, 0.4049390118493956]
['EX_hxan_e', 4.831239055186757, 0.38405123267181995]
['EX_urea_e', 6.360380469752657, 0.18985060211551838]
['inf']
EX_glyclt_e
['EX_enlipa_e', 0.021885753672477217, 0.0]
['EX_fad_e', 0.02240934914002436, 0.0]
['EX_meoh_e', 1.26426365302486e-07, 0.0]
['EX_pheme_e', 0.06793340199368081, 0.0]
['EX_eca4colipa_e', 0.009554795566696677, 0.0]
['EX_ins_e', 0.19391952309996752, 0.0]
['EX_phe__L_e', 0.38649504950499064, 0.0]
['EX_enter_e', 0.12392380952380944, 0.0]
['EX_feenter_e', 0.12392380952381724, 0.0]
['test', None, 0.0]
EX_acnam_e
['EX_lac__D_e', 0.9969570655185862, 0.6869109994270236]
['EX_etoh_e', 0.9802774926303073, 0.6869109994270169]
['EX_hxa_e', 0.36918991189148936, 0.6866161767023167]
['EX_pyr_e', 1.1844736255176258, 0.6864380589018443]
['

,0,1,2,3,4,5,6,7
0,carbon source ID,carbon source,simulation id,simulation,biomass,largest byproduct,byproduct id,amount of byproduct
1,EX_malt_e,Maltose exchange,WT KO,wild type,0.822023,Acetate exchange,EX_ac_e,2.343832
2,EX_malt_e,Maltose exchange,EX_ac_e KO,Acetate exchange,0.80387,Ethanol exchange,EX_etoh_e,1.822134
3,EX_malt_e,Maltose exchange,EX_etoh_e KO,Ethanol exchange,0.80387,D-lactate exchange,EX_lac__D_e,1.822134
4,EX_malt_e,Maltose exchange,EX_lac__D_e KO,D-lactate exchange,0.802803,Hexanoate (n-C6:0) exchange,EX_hxa_e,0.689033
...,...,...,...,...,...,...,...,...
89,EX_melib_e,Melibiose exchange,EX_succ_e KO,Succinate exchange,0.775058,4-Aminobutanoate exchange,EX_4abut_e,1.472704
90,EX_melib_e,Melibiose exchange,EX_4abut_e KO,4-Aminobutanoate exchange,0.772614,D-Alanine exchange,EX_ala__D_e,0.018761
91,EX_melib_e,Melibiose exchange,EX_ala__D_e KO,D-Alanine exchange,0.772477,5-Methylthio-D-ribose exchange,EX_5mtr_e,0.00521
92,EX_melib_e,Melibiose exchange,EX_5mtr_e KO,5-Methylthio-D-ribose exchange,0.0,Formate exchange,EX_for_e,0.001897


In [5]:
results_df.to_csv('byproduct_screen.csv')